In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
#

In [3]:
df=pd.read_csv('ds_salaries.csv')
df.head(10)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
5,2023,SE,FT,Applied Scientist,222200,USD,222200,US,0,US,L
6,2023,SE,FT,Applied Scientist,136000,USD,136000,US,0,US,L
7,2023,SE,FT,Data Scientist,219000,USD,219000,CA,0,CA,M
8,2023,SE,FT,Data Scientist,141000,USD,141000,CA,0,CA,M
9,2023,SE,FT,Data Scientist,147100,USD,147100,US,0,US,M


In [4]:
df.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,3755.000000,3.755000e+03,3755.000000,3755.000000
mean,2022.373635,1.906956e+05,137570.389880,46.271638
std,0.691448,6.716765e+05,63055.625278,48.589050
min,2020.000000,6.000000e+03,5132.000000,0.000000
25%,2022.000000,1.000000e+05,95000.000000,0.000000
50%,2022.000000,1.380000e+05,135000.000000,0.000000
75%,2023.000000,1.800000e+05,175000.000000,100.000000
max,2023.000000,3.040000e+07,450000.000000,100.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


In [6]:
df.nunique()

work_year                4
experience_level         4
employment_type          4
job_title               93
salary                 815
salary_currency         20
salary_in_usd         1035
employee_residence      78
remote_ratio             3
company_location        72
company_size             3
dtype: int64

In [7]:
df1=df.drop(['salary','salary_currency','remote_ratio','employee_residence'],axis=1)
df1

,work_year,experience_level,employment_type,job_title,salary_in_usd,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,85847,ES,L
1,2023,MI,CT,ML Engineer,30000,US,S
2,2023,MI,CT,ML Engineer,25500,US,S
3,2023,SE,FT,Data Scientist,175000,CA,M
4,2023,SE,FT,Data Scientist,120000,CA,M
...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,US,L
3752,2020,EN,FT,Data Scientist,105000,US,S
3753,2020,EN,CT,Business Data Analyst,100000,US,L


In [8]:
df1['work_year']=pd.factorize(df1['work_year'])[0]
df1['experience_level']=pd.factorize(df1['experience_level'])[0]
df1['employment_type']=pd.factorize(df1['employment_type'])[0]
df1['job_title']=pd.factorize(df1['job_title'])[0]
df1['company_location']=pd.factorize(df1['company_location'])[0]
df1['company_size']=pd.factorize(df1['company_size'])[0]



In [9]:
df1['work_year']=df1['work_year'].astype('category')
df1['experience_level']=df1['experience_level'].astype('category')
df1['employment_type']=df1['employment_type'].astype('category')
df1['job_title']=df1['job_title'].astype('category')
df1['company_location']=df1['company_location'].astype('category')
df1['company_size']=df1['company_size'].astype('category')

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   work_year         3755 non-null   category
 1   experience_level  3755 non-null   category
 2   employment_type   3755 non-null   category
 3   job_title         3755 non-null   category
 4   salary_in_usd     3755 non-null   int64   
 5   company_location  3755 non-null   category
 6   company_size      3755 non-null   category
dtypes: category(6), int64(1)
memory usage: 57.6 KB


In [11]:
df1

,work_year,experience_level,employment_type,job_title,salary_in_usd,company_location,company_size
0,0,0,0,0,85847,0,0
1,0,1,1,1,30000,1,1
2,0,1,1,1,25500,1,1
3,0,0,0,2,175000,2,2
4,0,0,0,2,120000,2,2
...,...,...,...,...,...,...,...
3750,2,0,0,2,412000,1,0
3751,3,1,0,0,151000,1,0
3752,2,2,0,2,105000,1,1
3753,2,2,1,20,100000,1,0


In [62]:
count=df[['company_size']].value_counts()
count

company_size
M               3153
L                454
S                148
dtype: int64

In [58]:
for var in df1.columns:
    print(df1[var].value_counts())

0    1785
1    1664
3     230
2      76
Name: work_year, dtype: int64
0    2516
1     805
2     320
3     114
Name: experience_level, dtype: int64
0    3718
3      17
1      10
2      10
Name: employment_type, dtype: int64
11    1040
2      840
4      612
9      289
7      103
      ... 
71       1
61       1
68       1
62       1
92       1
Name: job_title, Length: 93, dtype: int64
100000    99
150000    98
120000    91
160000    84
130000    82
          ..
234100     1
223800     1
172100     1
232200     1
94665      1
Name: salary_in_usd, Length: 1035, dtype: int64
1     3040
4      172
2       87
0       77
6       58
      ... 
34       1
47       1
46       1
45       1
71       1
Name: company_location, Length: 72, dtype: int64
2    3153
0     454
1     148
Name: company_size, dtype: int64


In [12]:
df1.describe()

,salary_in_usd
count,3755.000000
mean,137570.389880
std,63055.625278
min,5132.000000
25%,95000.000000
50%,135000.000000
75%,175000.000000
max,450000.000000


In [14]:
table=pd.crosstab(df['salary_in_usd'],df['company_size'],margins=True)
table

company_size,L,M,S,All
salary_in_usd,,,,
5132,0,1,0,1
5409,1,1,0,2
5679,0,0,1,1
5707,0,1,0,1
5723,0,0,1,1
...,...,...,...,...
423000,1,0,0,1
423834,1,0,0,1
430967,0,1,0,1


In [15]:
table1=pd.crosstab(df['work_year'],df['experience_level'],margins=True)
table1

experience_level,EN,EX,MI,SE,All
work_year,,,,,
2020,23,3,32,18,76
2021,55,10,92,73,230
2022,124,41,361,1138,1664
2023,118,60,320,1287,1785
All,320,114,805,2516,3755


in the year 2020 MI had the most number of work force follwed crosly by EN and then SE together with EX having the least.In 2021 MI was still ahead of all but this time followed by SE having overtaken EN by 18 point pushing EN to position three and again EX was at the last position with a small improvement.things changed very much in the year 2022 which saw SE make a very big jump to the top position with a wonderful improvement of over 1000 points with all other experience levels following from far.2023 was almost the same as 2022 but with small changes

In [23]:
table2=pd.crosstab(df1['employment_type'],df1['salary_in_usd'])
table2

salary_in_usd,5132,5409,5679,5707,5723,5882,6072,6270,6304,6359,...,376080,380000,385000,405000,412000,416000,423000,423834,430967,450000
employment_type,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,2,1,1,1,...,1,1,1,1,1,0,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# contingecy tables

In [24]:
table3 = pd.pivot_table(df1, index='experience_level', columns='employment_type', values='salary_in_usd', aggfunc=np.mean)
print(table3.head(30))

employment_type               0         1             2        3
experience_level                                                
0                 153214.454799   97500.0  53333.333333      NaN
1                 105321.489899   78600.0  41615.600000  42561.0
2                  80496.529801   65234.5  75000.000000  38885.0
3                 192974.566372  416000.0           NaN      NaN


In [65]:
table4 = pd.crosstab(df['work_year'],df['company_size'],values=df['salary_in_usd'], aggfunc='max',margins=True)
print(table4)

company_size       L       M       S     All
work_year                                   
2020          412000  450000  260000  450000
2021          423000  200000  416000  423000
2022          405000  430967  275000  430967
2023          423834  385000  200000  423834
All           423834  450000  416000  450000


  Reganding the highest payed salary within the three sizes of the companies  and a period of four years.
the year saw 2020, midium size companies paid the most followed by large campanies and the least was the small size companis.
however in the year 2021 small size companies showed improvement and paid more compared to midium sized companiesbut large size companies the highest.
2022 come with better payments within midium size companies,not far much from large size companies than small size campanies which show a reduced pay inthe same year.
the year 2023 was much better for large size companies follwed by midium size companies but wast for the small size companies.
in concrusion the payment with thelarge size companies is more stable than midium and small size companies but midium companies are have better stability than small size companies.


In [67]:
table5 = pd.crosstab(df['experience_level'],df['company_size'],values=df['salary_in_usd'], aggfunc='max',margins=True)
print(table5)

company_size           L       M       S     All
experience_level                                
EN                300000  220000  150000  300000
EX                325000  353200  416000  416000
MI                423000  450000  200000  450000
SE                423834  385000  275000  423834
All               423834  450000  416000  450000


In [17]:
table6=pd.crosstab(df['job_title'],df['company_size'],margins=True)
table6

company_size,L,M,S,All
job_title,,,,
3D Computer Vision Researcher,0,2,2,4
AI Developer,2,6,3,11
AI Programmer,1,1,0,2
AI Scientist,6,5,5,16
Analytics Engineer,1,101,1,103
...,...,...,...,...
Research Scientist,15,63,4,82
Software Data Engineer,1,1,0,2
Staff Data Analyst,0,1,0,1


large company size could have been expected to have the highest number of work force compared to midium and small size company but its the midium size company that has the highest. this may be atributed to the fact that the midium company size can comfotably handle all the operation within its level and at the same time be able to do part of what is being done by large size company as well as all that which is being done by the small size company

In [21]:
table7=pd.crosstab(df1['company_location'],df1['company_size'],margins=True)
table7

company_size,0,1,2,All
company_location,,,,
0,6,2,69,77
1,263,54,2723,3040
2,18,4,65,87
3,17,18,21,56
4,20,8,144,172
...,...,...,...,...
68,0,1,0,1
69,1,0,0,1
70,0,1,0,1


In [22]:
x=df1.drop(['salary_in_usd','company_size','employment_type','work_year'],axis=1)
y=df1['salary_in_usd']

In [45]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)


In [51]:
model=RandomForestRegressor(n_estimators=100,random_state=0)

In [52]:
model.fit(x_train,y_train)

RandomForestRegressor(random_state=0)

In [53]:
importances = model.feature_importances_
feature_importances = pd.DataFrame({'Feature':x_train.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=True)
print(feature_importances)

            Feature  Importance
0  experience_level    0.160895
1         job_title    0.305406
2  company_location    0.533700


In [54]:
y_pred=model.predict(x_test)
y_pred

array([155973.69209797, 109912.87841535, 128575.70611039, ...,
       162820.38547779, 122496.78571429, 122877.40946647])

In [55]:
mse=mean_squared_error(y_test,y_pred)
print('mean squared error:',mse)

mean squared error: 2658013176.0813217


In [56]:
r2=model.score(x_test,y_test)
print("R-squared:",r2)

R-squared: 0.3606891435903571


the variable in the data set where not sufficient to explane the variation in the saraly